In [1]:
# import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

import nltk
from nltk.stem import PorterStemmer
ps = nltk.PorterStemmer()
from nltk.stem import WordNetLemmatizer 
stemmer = WordNetLemmatizer()
stopwords =nltk.corpus.stopwords.words('english')

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import re
import string

import joblib


In [2]:
# import csv
full_data = pd.read_csv("mbti_1.csv")

# selecting random percentage of rows because of memory issues
data = full_data.sample(frac = .1)
data.columns = ['type', 'posts']

In [3]:
X = data['posts'].values
y = data['type'].values

In [4]:
posts = []

for text in range(0, len(X)):
    
    # This removes the special characters
    post = re.sub(r'\W', ' ', str(X[text]))
    
    # This removes multiple spaces
    post = re.sub(r'\s+', ' ', post, flags=re.I)
    
    # This converts all text to lower case
    post = post.lower()
    
    # Time to lemmatize
    post = post.split()

    post = [stemmer.lemmatize(word) for word in post]
    post = ' '.join(post)
    
    posts.append(post)

In [5]:
count_vect = CountVectorizer(max_features=1500, stop_words=stopwords)
X = count_vect.fit_transform(posts).toarray()

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y) 


In [7]:
rf_class = RandomForestClassifier(n_estimators=1000, random_state=0)
rf_class.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000, random_state=0)

In [8]:
y_pred = rf_class.predict(X_test)

In [9]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[ 0  0  0  0  0  0  0  2  4  0  0  0  0  0  0]
 [ 0  3  0  0  0  0  0  4  6  2  4  0  0  0  0]
 [ 0  0  0  0  0  0  0  3  2  3  3  0  0  0  0]
 [ 0  0  0  1  0  0  0  4  4  0  3  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  2  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  2  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 23  5  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  2 38  1  5  0  0  0  0]
 [ 0  0  0  0  0  0  0  2  6 17  7  0  0  0  0]
 [ 0  0  0  0  0  0  0  2  2  1 24  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  3  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  3  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  4  1  1  0  0  0  1]
 [ 0  0  0  0  0  0  0  1  7  1  3  0  0  0  0]]
              precision    recall  f1-score   support

        ENFJ       0.00      0.00      0.00         6
        ENFP       1.00      0.16      0.27        19
        ENTJ       0.00      0.00      0.00        11
        ENTP       1.00      0.08      0.15        12
        

C:\Users\ruhlk\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
import pickle


In [11]:
# yo uname the file here
 with open('mbti_model.pickle', 'wb') as f:
      pickle.dump(rf_class, f)
        
# with open('vector.pickle', 'wb') as f:
#     pickle.dump(count_vect,f)